In [1]:
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask_ml.model_selection import train_test_split

In [2]:
# Importation des `noms de métiers`
# Création du dico des `noms de métiers`
jobNames = pd.read_csv('./data/categories_string.csv')['0'].to_dict()


# Importation du jeu de données d'entrainement

trainX=(dd.from_pandas(pd.read_json('./data/train.json',dtype={'description':'string','gender':'string'})
          .set_index('Id'), npartitions=11))

trainY=dd.read_csv('./data/train_label.csv').set_index('Id',npartitions=11)

# Concatenation en un seul DataFrame pour visualiser
trainDF= trainX.assign(Y = trainY.Category).categorize(columns=['gender'])

In [3]:
print(jobNames)

{0: 'pastor', 1: 'model', 2: 'yoga_teacher', 3: 'teacher', 4: 'personal_trainer', 5: 'painter', 6: 'journalist', 7: 'interior_designer', 8: 'surgeon', 9: 'accountant', 10: 'dj', 11: 'physician', 12: 'comedian', 13: 'software_engineer', 14: 'nurse', 15: 'poet', 16: 'dentist', 17: 'chiropractor', 18: 'filmmaker', 19: 'professor', 20: 'photographer', 21: 'rapper', 22: 'psychologist', 23: 'paralegal', 24: 'architect', 25: 'composer', 26: 'attorney', 27: 'dietitian'}


In [4]:
trainY.head()

,Category
Id,
0,19
1,9
2,19
3,24
4,24


In [5]:
def macro_disparate_impact(DF):
    """Fonction Dask de calcul de la Fairness (disparate_impact)"""
    # "Jointure" sur les noms de métiers
    jobs=DF.Y.apply(lambda x :jobNames[x],meta=('Y', 'int'))
    
    # Concatenation métier | genre
    people=dd.concat([jobs,DF.gender], axis=1).reset_index()
    
    # Création de la table de comptage par genre
    counts = people.pivot_table(index='Y',columns='gender',aggfunc='count')
    
    # Calcul de l'indicateur
    # Paramètre axis=1 <== 'max/min par ligne'
    disparate_impact= (counts.max(axis=1)/counts.min(axis=1)).mean()
    
    return disparate_impact

In [6]:
macro_disparate_impact(trainDF).compute()

3.898171170378378

In [7]:
from dask_ml.feature_extraction.text import HashingVectorizer
import re

pd.set_option("display.max_colwidth", None)

def prepareTxt(txt):
    txt_clean=txt.lower()
    liste_regex=[(r"dr\.","doctor")]
    for regex,new in liste_regex: 
        txt_clean=re.sub(regex,new,txt_clean)
    return txt_clean

In [ ]:
from dask_ml.feature_extraction.text import HashingVectorizer
from dask_ml.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Mettre en minuscule
#

# Splitter en Apprentissage-Validation


XTrainSet= trainSet.loc[:,trainSet.columns!='Y']
XValidSet= validSet.loc[:,validSet.columns!='Y']

YTrainSet= trainSet['Y']
#.loc[:,trainSet.columns!='Y']
YValidSet= validSet['Y']
#.loc[:,validSet.columns!='Y']
#XvalidSet, YtrainSet,YvalidSet= train_test_split(trainX,trainY,test_size=0.3)





#vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False,n_features=1)

#X_train = vectorizer.fit_transform(XTrainSet['description'])
#X_valid = vectorizer.transform(XValidSet['description'])

In [76]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

LDA=LinearDiscriminantAnalysis()
NAIVEBAYES=MultinomialNB()
ada=AdaBoostClassifier(n_estimators=100)

gbm=GradientBoostingClassifier(n_estimators=100)
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf',gbm),
])


In [77]:
trainSet,validSet=train_test_split(trainDF,shuffle=True,test_size=0.45)

XTrainSet= trainSet.loc[:,trainSet.columns!='Y']
XValidSet= validSet.loc[:,validSet.columns!='Y']

YTrainSet= trainSet['Y']
#.loc[:,trainSet.columns!='Y']
YValidSet= validSet['Y']


In [78]:
# Fitting our train data to the pipeline
text_clf.fit(XTrainSet.description, YTrainSet)

KeyboardInterrupt: 

In [ ]:
predicted = text_clf.predict(XValidSet.description)

In [64]:
 predicted
#YValidSet

array([18, 26, 19, ..., 26, 19, 22])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predicted,YValidSet)

In [ ]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, train_data.target_names[category]))

In [ ]:
predicted = text_clf.predict(docs_test)

In [ ]:
from dask_ml.feature_extraction.text import CountVectorizer
from dask_ml.linear_model import LogisticRegression

from nltk.corpus import stopwords

vectorizer =CountVectorizer()
vectorizer.fit(trainDF['description'])

Train_X_vect = vectorizer.transform(XTrainSet)
Test_X_vect = vectorizer.transform(XTrainSet)

x_chunk = Train_X_vect.compute_chunk_sizes() 
y_chunk = y_train.compute_chunk_sizes() 

lr = LogisticRegression()
lr.fit(x_chunk,y_chunk) 

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline = make_pipeline(CountVectorizer(),
                         TfidfTransformer(),
                         SGDClassifier())

In [ ]:
from dask_ml.metrics import accuracy_score

predictions = model.predict(xValidSet)
accuracy_score(yValidSet, predictions)